In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import torch
import torchvision
import torchaudio
import os
from os.path import isfile, join, isdir, exists
import numpy as np
import matplotlib.pyplot as plt

In [3]:
torch.cuda.get_device_name(0)


'NVIDIA GeForce RTX 3080'

In [4]:
s_rate = 16000
t = 10

In [5]:
directory = "maestro-v3.0.0"
save_directory = "mel"
#if not exists(save_directory):
    
#    os.mkdir("./mel")
    
dirnames = [f for f in os.listdir(directory) if isdir(join(directory, f))]

for d in dirnames:
    if not exists(join(save_directory, d)):
        os.mkdir(join(save_directory, d))


fnames = []
for dirs in dirnames:
    for f in os.listdir(join(directory, dirs)):
        fnames.append(join(dirs, f))

for fname in fnames:
    f = fname
    fname = join(directory, fname)
    if(fname[-3:] != 'wav'):
        continue
    
    waveform, sample_rate = torchaudio.load(fname, normalize=True)
    waveform = torchaudio.functional.resample(waveform, sample_rate, s_rate)
    waveform = torch.mean(waveform, dim=0, keepdim=True)[0]
    for i in range(0, len(waveform), t*s_rate):
        if(i+t*s_rate > len(waveform)):
            break
        
        cur = waveform[i:i+t*s_rate]
        
        transform = torchaudio.transforms.MelSpectrogram(sample_rate=s_rate, n_fft=2048, win_length=2048, hop_length=160, n_mels=229, f_min=30, f_max=8000)
        mel_specgram = transform(cur) 
        log_ms = np.log(mel_specgram)
        
        torch.save(log_ms, save_directory+"/"+f[:-4]+"_"+str(i//(t*s_rate))+".pt")


  

/tmp/ipykernel_5905/3562131082.py:36: RuntimeWarning: divide by zero encountered in log
  log_ms = np.log(mel_specgram)


In [ ]:
import mido

directory = "maestro-v3.0.0"
save_directory = "mid"
#if not exists(save_directory):
    
#    os.mkdir("./mel")
    
dirnames = [f for f in os.listdir(directory) if isdir(join(directory, f))]

for d in dirnames:
    if not exists(join(save_directory, d)):
        os.mkdir(join(save_directory, d))


fnames = []
for dirs in dirnames:
    for f in os.listdir(join(directory, dirs)):
        fnames.append(join(dirs, f))

for fname in fnames:
    f = fname
    fname = join(directory, fname)
    if(fname[-4:] != 'midi'):
        continue
    
    mid = mido.MidiFile(fname, clip=True)
    
    mlen = mid.length
    mticks = mid.ticks_per_beat
    #print(mticks)
    out = np.zeros((88, mido.second2tick(mlen)))
    indxs = np.zeros(88)
    
    
    for msg in mid.track[1]:
        note = msg.note
        time = msg.time
        on_value = 1 if msg.velocity != 0 else 0
        
        
    '''
    for i in range(0, len(waveform), t*s_rate):
        if(i+t*s_rate > len(waveform)):
            break
        
        cur = waveform[i:i+t*s_rate]
        
        
        torch.save(log_ms, save_directory+"/"+f[:-4]+"_"+str(i//(t*s_rate))+".pt")

    '''
    if(mticks != 384):
        print("dogg", mticks)